In [ ]:
from typing import Tuple

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap

from simulation.loader import load_environment, TileType


In [ ]:
def plot_line_of_sight(environment: np.ndarray, mid_pos: Tuple[float, float]) -> Tuple[plt.Figure, plt.Axes]:
    mid_pos = np.array(mid_pos)
    visibility = np.zeros_like(environment)

    n_r = 85
    n_d = 3 * 6
    x, y = mid_pos
    for r in np.linspace(0, 2 * np.pi, n_r, endpoint=False):
        for d in np.linspace(0, 3 * 4 - 1 / 4, n_d):
            dx, dy = np.sin(r) * d, np.cos(r) * d
            if environment[int(x + dx + 0.5), int(y + dy + 0.5)] == TileType.WALL:
                break
            visibility[int(x + dx + 0.5), int(y + dy + 0.5)] = 1


    im = np.zeros_like(environment)
    im[environment == TileType.WALL] = 1
    im[visibility == 1] = 2


    cmap = ListedColormap([
        [0.5, 0.5, 0.5, 1],
        [0.3, 0.3, 0.35, 1],
        [1, 1, 1, 1],
    ])

    fig, ax = plt.subplots(figsize=(4, 4))
    ax.axis("off")
    ax.set_aspect("equal")
    ax.imshow(im.T, cmap=cmap)
    ax.plot(*mid_pos, "ko")

    for r in np.linspace(0, 2 * np.pi, n_r, endpoint=False):
        ray_pos = []
        v = True
        for d in np.linspace(0, 3 * 4 - 1 / 4, n_d):
            dx, dy = np.sin(r) * d, np.cos(r) * d
            if environment[int(x + dx + 0.5), int(y + dy + 0.5)] == TileType.WALL:
                v = False
            ray_pos.append((x + dx, y + dy, v))
        xs, ys, vis = map(np.array, zip(*ray_pos))

        # Rays
        plt.plot(xs[vis], ys[vis], ".-", color="darkgreen", alpha=0.2)

        # Last point
        v_idx, = np.where(vis == True)
        v_last = v_idx[-1]
        color = "red" if v_last != xs.shape[0] - 1 else "blue"
        plt.plot(xs[v_last], ys[v_last], ".-", color=color)

    fig.tight_layout()
    return fig, ax


environment = load_environment("../data/chalmers_karresturang_normal.txt", 3)
environment = environment[-26:, 65:65 + 26]
plot_line_of_sight(environment, (12.5, 12.5))
plt.savefig("line-of-sight_report.pdf", dpi=300)
plt.show()

In [ ]:
def plot_risk_of_infection_model() -> Tuple[plt.Figure, plt.Axes]:
    x = np.linspace(0, 5.5)
    risk = 1 - x/4
    risk[risk <= 0] = 0

    fig, ax = plt.subplots(figsize=(7 * 2 / 3, 2))

    ax.set_xlim((-0.5, 5.5))
    ax.set_yticks([])
    ax.set_xlabel("Distance from infected person [m]"); ax.set_ylabel("Risk of infection")
    ax.set_aspect("equal")

    ax.axvline(0, ls="--", c="k")
    ax.axhline(0, ls="-", c="k")
    ax.plot(x, 2 * risk, "r-")

    fig.tight_layout()
    return fig, ax

plot_risk_of_infection_model()
plt.savefig("risk-distance_report.pdf", dpi=300)
plt.show()